In [1]:
'''
Steps to get EDBOplus working:
1)Create and activate a conda environment
2)Download and navigate to the edbopaper directory in a terminal window (or Anaconda prompt window) and pip install -r requirements.txt
3)Put this .ipynb file in ANY directory (doesn't matter where it is).
4)Set the path to edbopaper in sys.path.append below
5)...
6)Profit.

'''

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


sys.path.append('/home/sanjay/AFRL/git_edbo/edbopaper')

In [2]:
#import EDBOplus
from plus.optimizer_botorch import EDBOplus

In [3]:
#setting up reaction components
#np.arange creates a list of numbers from (start,stop,stepsize)
components = {
              'temperature':np.arange(30,140,3).tolist(),   # Discrete grid of concentrations
              'time': np.arange(1,45,2).tolist(),
              'stoichiometry': np.arange(0.33,0.66,0.025).tolist()}

In [4]:
#need to generage the data scope first
# scope = EDBOplus.generate_reaction_scope(components=components)

In [5]:
#Initialize EDBOplus class that will store predicted means, variances, etc.
bo = EDBOplus()

args = {'objectives': ['production_rate_(g/hr)','yield'], 'objective_mode': ['max','max'], 'batch': 3, 'seed': 42}


In [6]:
#Run this cell on scope data (csv file called reaction.csv)
bo.run(**args)

Using EHVI acquisition function.
Using hyperparameters optimized for continuous variables.
Using hyperparameters optimized for continuous variables.
Number of QMC samples: 128
Acquisition function optimized.
Predictions obtained and expected improvement obtained.


,temperature,time,stoichiometry,production_rate_(g/hr),yield,priority
15,138,43,0.330,PENDING,PENDING,1.0
155,138,21,0.630,PENDING,PENDING,1.0
605,132,43,0.655,PENDING,PENDING,1.0
3,138,43,0.655,PENDING,PENDING,0.0
4,138,43,0.630,PENDING,PENDING,0.0
...,...,...,...,...,...,...
11391,78,23,0.480,3.5,65,-1.0
11392,54,41,0.580,0.6,41,-1.0
11393,54,41,0.480,3.4,66,-1.0
11394,45,29,0.605,0.4,35,-1.0


In [7]:
#Update reaction.csv with response values for highest priority samples (or update other priority reactions if you're a rebel and like to do what you want)

#Re-run the above cell to train the model and suggest next experiments 

In [8]:
#convert csv into df

#df of reaction data
df = pd.read_csv('reaction.csv')
#df of reaction data + predictions
df2 = pd.read_csv('pred_reaction.csv')



In [9]:
#Create a df to plot points 

#turn all pending data to NaN
df = df.replace('PENDING',pd.NaT)
#delete all the NaN data points 
df = df.dropna(axis=0)
#turn data from string to numeric (the pending made them strings)
df['production_rate_(g/hr)'] = df['production_rate_(g/hr)'].apply(pd.to_numeric)
df['yield'] = df['yield'].apply(pd.to_numeric)
#length of results
results_length = []
#loop through all results (-1 means it is a result)
for i in range(len(df['priority']==-1)):
    results_length.append(i)
#max value of yield and production
max_value_yield = df['yield'].max()
max_value_production_rate = df['production_rate_(g/hr)'].max()

#convert to numpy array for plotting in plotly
pareto_x = df['yield'].to_numpy()
pareto_y =df['production_rate_(g/hr)'].to_numpy()



In [46]:
#plotly scatter plot with pareto front

def plot_pareto_frontier(Xs, Ys, maxX=True, maxY=True):
    '''Pareto frontier selection process'''
    sorted_list = sorted([[Xs[i], Ys[i]] for i in range(len(Xs))], reverse=maxY)
    pareto_front = [sorted_list[0]]
    for pair in sorted_list[1:]:
        if maxY:
            if pair[1] >= pareto_front[-1][1]:
                pareto_front.append(pair)
        else:
            if pair[1] <= pareto_front[-1][1]:
                pareto_front.append(pair)
    
    '''Plotting process'''
    fig = px.scatter(x = Xs,y= Ys,
     title= 'Multiobjective Optimization of a Continuous Flow Reaction',
     labels={
         'x':'Yield (%)',
         'y':'Production Rate (g/hr)'
     },)
    pf_X = [pair[0] for pair in pareto_front]
    pf_Y = [pair[1] for pair in pareto_front]
    fig.add_trace(go.Scatter(x=pf_X, y=pf_Y, name= 'Pareto Front'))
    fig.update_layout(
    title={
        'text': "Pareto Front for the Multiobjective Optimization of a Continuous Flow Reaction",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        )
    fig.show()

    #Figure 2, only the selected points for the pareto front
    fig2 = px.scatter(x =pf_X,y=pf_Y,labels={'x':'Yield (%)','y':'Production Rate (g/hr)'})
    fig2.update_layout(
    title={
        'text': "Pareto Front Points",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        title_y=0.95
        )
    fig2.show()

    #figure 3, Scatter plot of all data points with hover to see inputs and bigger sized points
    fig3 = px.scatter(df,x='yield', y='production_rate_(g/hr)',
    size='yield',color='yield',
    hover_data=['temperature','time','stoichiometry'],
    labels={'yield':'Yield (%)','production_rate_(g/hr)':'Production Rate (g/hr)'})
    fig3.update_layout(
        title={
            'text': "Multiobjective Optimization of a Continuous Flow Reaction",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            title_y=0.95
            )      
    fig3.show()
    print('Highest yield is {}%'.format(df['yield'].max()))
    print('Highest production rate is {}g/hr'.format(df['production_rate_(g/hr)'].max()))

In [47]:
#View plots
plot_pareto_frontier(pareto_x, pareto_y)

Highest yield is 88%
Highest production rate is 6.5g/hr
